In [ ]:
# Set some environment variables
import os
gpu_num = "" # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
import warnings
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Other imports
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera

# Fix the seed for reproducible results
tf.random.set_seed(1)

In [ ]:
from sionna.rt.antenna import *
def bf_pattern(theta, phi, slant_angle=0.0,
                    polarization_model=2, dtype=tf.complex64):
    r"""
    Antenna pattern from 3GPP TR 38.901 (Table 7.3-1) [TR38901]_

    Input
    -----
    theta: array_like, float
        Zenith angles wrapped within [0,pi] [rad]

    phi: array_like, float
        Azimuth angles wrapped within [-pi, pi) [rad]

    slant_angle: float
        Slant angle of the linear polarization [rad].
        A slant angle of zero means vertical polarization.

    polarization_model: int, one of [1,2]
        Polarization model to be used. Options `1` and `2`
        refer to :func:`~sionna.rt.antenna.polarization_model_1`
        and :func:`~sionna.rt.antenna.polarization_model_2`,
        respectively.
        Defaults to `2`.

    dtype : tf.complex64 or tf.complex128
        Datatype.
        Defaults to `tf.complex64`.

    Output
    ------
    c_theta: array_like, complex
        Zenith pattern

    c_phi: array_like, complex
        Azimuth pattern

    """
    rdtype = dtype.real_dtype
    theta = tf.cast(theta, rdtype)
    phi = tf.cast(phi, rdtype)

    slant_angle = tf.cast(slant_angle, rdtype)
    M = 8
    N = 8
    da = 0.5
    theta_p = -PI/6#0.0
    phi_p = 0.0#-PI/3

    #wrap theta_p to [-PI/2, PI/2]
    theta = theta+PI/2

    # Wrap phi to [-PI,PI]
    phi = tf.math.floormod(phi+PI, 2*PI)-PI

    if not theta.shape==phi.shape:
        raise ValueError("theta and phi must have the same shape.")
    if polarization_model not in [1,2]:
        raise ValueError("polarization_model must be 1 or 2")
    a_azi = tf.zeros_like(phi)

    phi = tf.where(phi<-PI/2, 0.0, phi)
    phi = tf.where(phi>PI/2, 0.0, phi)

    a_azi = tf.math.divide_no_nan(tf.sin( M * (PI * da * tf.sin(phi) - PI*da * tf.sin(phi_p) ) ) , (M * tf.sin( PI * da * tf.sin(phi) - PI * da * tf.sin(phi_p) )))
    a_azi = tf.where(phi<-PI/2, 0.0, a_azi)
    a_azi = tf.where(phi>PI/2, 0.0, a_azi)
    
    a_ele = tf.math.divide_no_nan(tf.sin( N * (PI * da * tf.sin(theta) - PI*da * tf.sin(theta_p) ) ), (N * tf.sin( PI * da * tf.sin(theta) - PI * da * tf.sin(theta_p) )))

    c = tf.complex(a_ele, a_azi)
    if polarization_model==1:
        return polarization_model_1(c, theta, phi, slant_angle)
    else:
        return polarization_model_2(c, slant_angle)

In [ ]:
# Prepare training data
from sionna.rt.antenna import hw_dipole_pattern, tr38901_pattern, visualize,iso_pattern
theta_train_vertical = np.linspace(0, np.pi, 10000)  # Your theta values
phi_train_vertical = tf.zeros_like(theta_train_vertical)  # Your phi values
c_thetas, c_phis = iso_pattern(theta_train_vertical, phi_train_vertical)
gains_train_vertical = np.abs(c_thetas)**2 + np.abs(c_phis)**2


# Prepare training data
phi_train_h = tf.constant(np.linspace(-np.pi, np.pi, 10000),tf.float32)  # Your theta values
theta_train_h = np.pi/2*tf.ones_like(phi_train_h)
c_thetas, c_phis = iso_pattern(theta_train_h, phi_train_h)
gains_train_h = np.abs(c_thetas)**2 + np.abs(c_phis)**2

gains_train = np.concatenate([gains_train_vertical, gains_train_h])
theta_train = np.concatenate([theta_train_vertical, theta_train_h])
phi_train = np.concatenate([phi_train_vertical, phi_train_h])

In [ ]:
X = np.array([theta_train,phi_train]).T
y = gains_train

In [ ]:
y

In [ ]:
X

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Neural Network Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(2,), activation='relu'),  # Increase neurons
    tf.keras.layers.Dense(256, activation='relu'),  # Deeper network
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Add dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for regression (gain prediction)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=16)

# Step 3: Evaluation
# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}')

# Predict on the test data
y_pred = model.predict(X_test)


In [ ]:
y_pred

In [ ]:

# Step 4: Visualization
# Plotting the true vs predicted gain for the test set
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', label='Predicted vs True')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', label='Ideal Fit')
plt.xlabel('True Gain')
plt.ylabel('Predicted Gain')
plt.title('True vs Predicted Gain')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sionna.constants import PI
import tensorflow as tf
from collections.abc import Sequence

def visualize_custom(pattern):
    r"""visualize(pattern)
    Visualizes an antenna pattern

    This function visualizes an antenna pattern with the help of three
    figures showing the vertical and horizontal cuts as well as a
    three-dimensional visualization of the antenna gain.

    Input
    -----
    pattern : callable
        A callable that takes as inputs vectors of zenith and azimuth angles
        of the same length and returns for each pair the corresponding zenith
        and azimuth patterns.

    Output
    ------
     : :class:`matplotlib.pyplot.Figure`
        Vertical cut of the antenna gain

     : :class:`matplotlib.pyplot.Figure`
        Horizontal cut of the antenna gain

     : :class:`matplotlib.pyplot.Figure`
        3D visualization of the antenna gain

    Examples
    --------
    >>> fig_v, fig_h, fig_3d = visualize(hw_dipole_pattern)

    .. figure:: ../figures/pattern_vertical.png
        :align: center
        :scale: 80%
    .. figure:: ../figures/pattern_horizontal.png
        :align: center
        :scale: 80%
    .. figure:: ../figures/pattern_3d.png
        :align: center
        :scale: 80%
    """
    # Vertical cut
    theta = np.linspace(0.0, PI, 1000)
    X = np.array([theta, np.zeros_like(theta)]).T
    g = pattern.predict(X)
    g = np.where(g==0, 1e-12, g)
    g_db = 10*np.log10(g)
    g_db_max = np.max(g_db)
    g_db_min = np.min(g_db)
    if g_db_min==g_db_max:
        g_db_min = -30
    else:
        g_db_min = np.maximum(-60., g_db_min)
    fig_v = plt.figure()
    plt.polar(theta, g_db)
    fig_v.axes[0].set_rmin(g_db_min)
    fig_v.axes[0].set_rmax(g_db_max+3)
    fig_v.axes[0].set_theta_zero_location("N")
    fig_v.axes[0].set_theta_direction(-1)
    plt.title(r"Vertical cut of the radiation pattern $G(\theta,0)$ ")

    # Horizontal cut
    phi = np.linspace(-PI, PI, 1000)
    X = np.array([PI/2*tf.ones_like(phi) ,tf.constant(phi, tf.float32)]).T
    g = pattern.predict(X)
    g = np.where(g==0, 1e-12, g)
    g_db = 10*np.log10(g)
    g_db_max = np.max(g_db)
    g_db_min = np.min(g_db)
    if g_db_min==g_db_max:
        g_db_min = -30
    else:
        g_db_min = np.maximum(-60., g_db_min)

    fig_h = plt.figure()
    plt.polar(phi, g_db)
    fig_h.axes[0].set_rmin(g_db_min)
    fig_h.axes[0].set_rmax(g_db_max+3)
    fig_h.axes[0].set_theta_zero_location("E")
    plt.title(r"Horizontal cut of the radiation pattern $G(\pi/2,\varphi)$")

    # # 3D visualization
    # theta = np.linspace(0.0, PI, 50)
    # phi = np.linspace(-PI, PI, 50)
    # theta_grid, phi_grid = np.meshgrid(theta, phi, indexing='ij')
    # X = np.array([theta_grid, phi_grid]).T
    # g = pattern.predict(X)
    # x = g * np.sin(theta_grid) * np.cos(phi_grid)
    # y = g * np.sin(theta_grid) * np.sin(phi_grid)
    # z = g * np.cos(theta_grid)

    # print(np.shape(z))

    # g = np.maximum(g, 1e-5)
    # g_db = 10*np.log10(g)

    def norm(x, x_max, x_min):
        """Maps input to [0,1] range"""
        x = 10**(x/10)
        x_max = 10**(x_max/10)
        x_min = 10**(x_min/10)
        if x_min==x_max:
            x = np.ones_like(x)
        else:
            x -= x_min
            x /= np.abs(x_max-x_min)
        return x

    # g_db_min = np.min(g_db)
    # g_db_max = np.max(g_db)

    # fig_3d = plt.figure()
    # ax = fig_3d.add_subplot(1,1,1, projection='3d')
    # ax.plot_surface(x, y, z, rstride=1, cstride=1, linewidth=0,
    #                 antialiased=False, alpha=0.7,
    #                 facecolors=cm.turbo(norm(g_db, g_db_max, g_db_min)))

    # sm = cm.ScalarMappable(cmap=plt.cm.turbo)
    # sm.set_array([])
    # cbar = plt.colorbar(sm, ax=ax, orientation="vertical", location="right",
    #                     shrink=0.7, pad=0.15)
    # xticks = cbar.ax.get_yticks()
    # xticklabels = cbar.ax.get_yticklabels()
    # xticklabels = g_db_min + xticks*(g_db_max-g_db_min)
    # xticklabels = [f"{z:.2f} dB" for z in xticklabels]
    # cbar.ax.set_yticks(xticks)
    # cbar.ax.set_yticklabels(xticklabels)

    # ax.view_init(elev=30., azim=-45)
    # plt.xlabel("x")
    # plt.ylabel("y")
    # ax.set_zlabel("z")
    # plt.suptitle(
    #     r"3D visualization of the radiation pattern $G(\theta,\varphi)$")

    return fig_v, fig_h, 1


visualize_custom(model)

In [ ]:
visualize(iso_pattern)

## Fourier expansion AntennaPattern

### version 1

In [ ]:
import numpy as np
import tensorflow as tf
from sionna.rt import polarization_model_1, polarization_model_2

class FourierTrainableAntennaPattern(tf.keras.layers.Layer):
    def __init__(self, num_terms, slant_angle=0.0, polarization_model=2, dtype=tf.complex64):
        super(FourierTrainableAntennaPattern, self).__init__()
        self._num_terms = num_terms
        self._polarization_model = polarization_model
        self._dtype = dtype
        self._rdtype = dtype.real_dtype
        self._slant_angle = tf.cast(slant_angle, self._rdtype)

    def build(self, input_shape):
        self._alpha = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms, self._num_terms)), dtype=self._rdtype)
        self._beta = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms, self._num_terms)), dtype=self._rdtype)
        self._gamma = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms, self._num_terms)), dtype=self._rdtype)
        self._delta = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms, self._num_terms)), dtype=self._rdtype)
        self._e_rad = tf.Variable((1), dtype=self._rdtype)

    def call(self, theta, phi):
        theta = tf.cast(theta, self._rdtype)
        phi = tf.cast(phi, self._rdtype)

        # Fourier series expansion
        gain = tf.zeros_like(theta, dtype=self._rdtype)

        for n in range(1, self._num_terms + 1):
            for m in range(1, self._num_terms + 1):
                gain += self._alpha[n-1, m-1] * tf.cos(n * theta) * tf.cos(m * phi)
                gain += self._beta[n-1, m-1] * tf.cos(n * theta) * tf.sin(m * phi)
                gain += self._gamma[n-1, m-1] * tf.sin(n * theta) * tf.cos(m * phi)
                gain += self._delta[n-1, m-1] * tf.sin(n * theta) * tf.sin(m * phi)

        gain *= self._e_rad

        c = tf.complex(gain, tf.zeros_like(gain))

        if self._polarization_model == 1:
            value1, value2 = polarization_model_1(c, theta, phi, self._slant_angle)
        else:
            value1, value2 = polarization_model_2(c, self._slant_angle)

        return value1, value2


### version 2

In [ ]:
import numpy as np
import tensorflow as tf
from sionna.rt import polarization_model_1, polarization_model_2
from sionna.constants import PI
from sionna.rt.utils import r_hat, normalize
from sionna.rt.antenna import hw_dipole_pattern, tr38901_pattern

class FourierTrainableAntennaPattern(tf.keras.layers.Layer):
    def __init__(self, num_terms, slant_angle=0.0, polarization_model=2, dtype=tf.complex64):
        super(FourierTrainableAntennaPattern, self).__init__()
        self._num_terms = num_terms
        self._polarization_model = polarization_model
        self._dtype = dtype
        self._rdtype = dtype.real_dtype
        self._slant_angle = tf.cast(slant_angle, self._rdtype)

    def build(self, input_shape):
        self._cos_coeffs_theta = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms,)), dtype=self._rdtype)
        self._sin_coeffs_theta = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms,)), dtype=self._rdtype)
        self._cos_coeffs_phi = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms,)), dtype=self._rdtype)
        self._sin_coeffs_phi = tf.Variable(tf.initializers.GlorotUniform()(shape=(self._num_terms,)), dtype=self._rdtype)
        self._e_rad = tf.Variable((1), dtype=self._rdtype)

    def call(self, theta, phi):
        theta = tf.cast(theta, self._rdtype)
        phi = tf.cast(phi, self._rdtype)

        # Fourier series expansion
        theta_terms = tf.zeros_like(theta, dtype=self._rdtype)
        phi_terms = tf.zeros_like(phi, dtype=self._rdtype)

        for n in range(1, self._num_terms + 1):
            theta_terms += self._cos_coeffs_theta[n-1] * tf.cos(n * theta) + self._sin_coeffs_theta[n-1] * tf.sin(n * theta)
            phi_terms += self._cos_coeffs_phi[n-1] * tf.cos(n * phi) + self._sin_coeffs_phi[n-1] * tf.sin(n * phi)

        gain = tf.sqrt(theta_terms**2 + phi_terms**2)
        gain *= self._e_rad

        c = tf.complex(gain, tf.zeros_like(gain))

        if self._polarization_model == 1:
            value1, value2 = polarization_model_1(c, theta, phi, self._slant_angle)
        else:
            value1, value2 = polarization_model_2(c, self._slant_angle)

        return value1, value2


### Fake dataset

### Exp dataset

In [ ]:
from scipy.interpolate import interp1d
# Define the model
num_terms = 50  # Example number
model = FourierTrainableAntennaPattern(num_terms=num_terms)

# Original vertical and horizontal datasets
theta_train_vertical = tf.constant([x * np.pi / 180 for x in horizontal_data["x"]], tf.float32)
phi_train_vertical = tf.zeros_like(theta_train_vertical)
gains_train_vertical = tf.constant(horizontal_data["y"], tf.float32)

phi_train_horizontal = tf.constant([x * np.pi / 180 for x in vertical_data["x"]], tf.float32)
theta_train_horizontal = (np.pi / 2) * tf.ones_like(phi_train_horizontal)
gains_train_horizontal = tf.constant(vertical_data["y"], tf.float32)

# Interpolation function
def interpolate_data(x, y, num_points, noise_level=1):
    x = np.array(x)  # Convert to NumPy array
    y = np.array(y)  # Convert to NumPy array
    f = interp1d(x, y, kind='linear')
    x_new = np.linspace(x.min(), x.max(), num_points)
    y_new = f(x_new)
    
    # Generate noise and add it to the interpolated values
    noise = np.random.normal(0, noise_level, y_new.shape)
    y_new_noisy = y_new + noise
    return x_new, y_new_noisy

# Interpolate vertical dataset
num_new_points_vertical = 10000
x_new_vertical, y_new_vertical = interpolate_data(horizontal_data["x"], horizontal_data["y"], num_new_points_vertical)
theta_new_vertical = tf.constant([x * np.pi / 180 for x in x_new_vertical], tf.float32)
phi_new_vertical = tf.zeros_like(theta_new_vertical)
gains_new_vertical = tf.constant(y_new_vertical, tf.float32)

# Interpolate horizontal dataset
num_new_points_horizontal = 10000
x_new_horizontal, y_new_horizontal = interpolate_data(vertical_data["x"], vertical_data["y"], num_new_points_horizontal)
phi_new_horizontal = tf.constant([x * np.pi / 180 for x in x_new_horizontal], tf.float32)
theta_new_horizontal = (np.pi / 2) * tf.ones_like(phi_new_horizontal)
gains_new_horizontal = tf.constant(y_new_horizontal, tf.float32)

# Combine the interpolated datasets
theta_train = tf.concat([theta_train_vertical, theta_new_vertical, theta_train_horizontal, theta_new_horizontal], axis=0)
phi_train = tf.concat([phi_train_vertical, phi_new_vertical, phi_train_horizontal, phi_new_horizontal], axis=0)
gains_train = tf.concat([gains_train_vertical, gains_new_vertical, gains_train_horizontal, gains_new_horizontal], axis=0)

In [ ]:
plt.polar(phi_train_horizontal,gains_train_horizontal, label='fake data')
plt.polar(phi_new_horizontal, gains_new_horizontal, label='real data', linestyle='--')

In [ ]:
plt.polar(theta_new_vertical,gains_new_vertical, label='fake data')
plt.polar(theta_train_vertical, gains_train_vertical, label='real data', linestyle='--')

In [ ]:
np.shape(gains_train)

## Spherical Gaussian pattern

In [ ]:
import tensorflow as tf
from sionna.rt import polarization_model_1, polarization_model_2
from sionna.constants import PI
from sionna.rt.utils import r_hat, normalize
from sionna.rt.antenna import hw_dipole_pattern

class TrainableAntennaPattern(tf.keras.layers.Layer):
    def __init__(self, num_mixtures, slant_angle=0.0, polarization_model=2, dtype=tf.complex64):
        super(TrainableAntennaPattern, self).__init__()
        self._num_mixtures = num_mixtures
        self._polarization_model = polarization_model
        self._dtype = dtype
        self._rdtype = dtype.real_dtype
        self._slant_angle = tf.cast(slant_angle, self._rdtype)

    def build(self, input_shape):
        self._mu = tf.Variable(tf.initializers.GlorotUniform()(shape=(1, self._num_mixtures, 3)), dtype=self._rdtype)
        self._lambdas = tf.Variable(tf.initializers.GlorotUniform()(shape=(1, self._num_mixtures)), dtype=self._rdtype)
        self._weights = tf.Variable(tf.initializers.GlorotUniform()(shape=(1, self._num_mixtures)), dtype=self._rdtype)
        self._e_rad = tf.Variable((1), dtype=self._rdtype)

    def call(self, theta, phi):
        theta = tf.cast(theta, self._rdtype)
        phi = tf.cast(phi, self._rdtype)
        v = tf.expand_dims(r_hat(theta, phi), -2)
        mu, _ = normalize(self._mu)
        lambdas = tf.abs(self._lambdas)
        a = lambdas / tf.cast(2*PI, self._rdtype) / (1 - tf.exp(-tf.cast(2, self._rdtype) * lambdas))
        gains = tf.cast(4*PI, self._rdtype) * a * tf.exp(lambdas * (tf.reduce_sum(v * mu, axis=-1) - tf.cast(1, self._rdtype)))
        weights = tf.nn.softmax(self._weights)
        gain = tf.reduce_sum(gains * weights, axis=-1)
        gain *= self._e_rad
        c = tf.complex(tf.sqrt(gain), tf.zeros_like(gain))
        if self._polarization_model == 1:
            value1, value2 = polarization_model_1(c, theta, phi, self._slant_angle)
        else:
            value1, value2 = polarization_model_2(c, self._slant_angle)
        return value1, value2

# Define the model
num_mixtures = 100  # Example number
model = TrainableAntennaPattern(num_mixtures=num_mixtures)

# Prepare training data
theta_train = np.linspace(0, np.pi, 10000)  # Your theta values
phi_train = tf.zeros_like(theta_train)  # Your phi values
c_thetas, c_phis = hw_dipole_pattern(theta_train, phi_train)
gains_train = np.abs(c_thetas)**2 + np.abs(c_phis)**2

## Training the model

In [ ]:
# Loss function
def loss_fn(model, theta, phi, true_gains):
    value1, value2 = model(theta, phi)
    predicted_gains = tf.square(tf.abs(value1)) + tf.square(tf.abs(value2))
    return tf.reduce_mean(tf.square(predicted_gains - true_gains))

# Optimizer
optimizer = tf.keras.optimizers.legacy.Adam()

# Training step
@tf.function
def train_step(model, theta, phi, true_gains):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, theta, phi, true_gains)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Function to get predicted gains
def get_predicted_gain(model, theta, phi):
    value1, value2 = model(theta, phi)
    predicted_gain = tf.square(tf.abs(value1)) + tf.square(tf.abs(value2))
    return predicted_gain

# Training loop
num_epochs = 10000  # Example number of epochs
for epoch in range(num_epochs):
    loss = train_step(model, theta_train, phi_train, gains_train)
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sionna.constants import PI
import tensorflow as tf

def visualize_horizzontal(pattern, second_pattern=None):
    """
    Visualizes an antenna pattern with the help of vertical cut.

    Parameters:
    pattern : callable
        A callable that takes as inputs vectors of zenith and azimuth angles
        of the same length and returns for each pair the corresponding zenith
        and azimuth patterns.
    second_pattern : callable, optional
        A second callable pattern to be plotted on the same figure.

    Returns:
    fig_v : matplotlib.pyplot.Figure
        Vertical cut of the antenna gain.
    """
    # Horizontal cut
    phi = np.linspace(-PI, PI, 1000)
    c_theta, c_phi = pattern(PI/2*tf.ones_like(phi) ,
                             tf.constant(phi, tf.float32))
    c_theta = c_theta.numpy()
    c_phi = c_phi.numpy()
    g = np.abs(c_theta)**2 + np.abs(c_phi)**2
    g = np.where(g==0, 1e-12, g)
    g_db = 10*np.log10(g)
    g_db_max = np.max(g_db)
    g_db_min = np.min(g_db)
    if g_db_min==g_db_max:
        g_db_min = -30
    else:
        g_db_min = np.maximum(-60., g_db_min)

    fig_h = plt.figure()
    plt.polar(phi, g_db, label='Trained pattern')
    if second_pattern is not None:
        c_theta, c_phi = second_pattern(PI/2*tf.ones_like(phi) ,
                             tf.constant(phi, tf.float32))
        c_theta = c_theta.numpy()
        c_phi = c_phi.numpy()
        g = np.abs(c_theta)**2 + np.abs(c_phi)**2
        g = np.where(g==0, 1e-12, g)
        g_db = 10*np.log10(g)
        g_db_max = np.max(g_db)
        g_db_min = np.min(g_db)
        if g_db_min==g_db_max:
            g_db_min = -30
        else:
            g_db_min = np.maximum(-60., g_db_min)
        plt.polar(phi, g_db, label='Test pattern', linestyle='--')
    fig_h.axes[0].set_rmin(-50)
    fig_h.axes[0].set_rmax(10)
    fig_h.axes[0].set_theta_zero_location("E")
    plt.title(r"Horizontal cut of the radiation pattern $G(\pi/2,\varphi)$")
    plt.legend()
    return fig_h

def visualize_vertical(pattern, second_pattern=None):
    """
    Visualizes an antenna pattern with the help of vertical cut.

    Parameters:
    pattern : callable
        A callable that takes as inputs vectors of zenith and azimuth angles
        of the same length and returns for each pair the corresponding zenith
        and azimuth patterns.
    second_pattern : callable, optional
        A second callable pattern to be plotted on the same figure.

    Returns:
    fig_v : matplotlib.pyplot.Figure
        Vertical cut of the antenna gain.
    """

    # Vertical cut
    theta = np.linspace(0.0, PI, 1000)
    c_theta, c_phi = pattern(theta, np.zeros_like(theta))
    g = np.abs(c_theta) ** 2 + np.abs(c_phi) ** 2
    g = np.where(g == 0, 1e-12, g)
    g_db = 10 * np.log10(g)
    g_db_max = np.max(g_db)
    g_db_min = np.min(g_db)
    if g_db_min == g_db_max:
        g_db_min = -30
    else:
        g_db_min = np.maximum(-60., g_db_min)

    fig_v = plt.figure()
    plt.polar(theta, g_db, label='Trained pattern')

    if second_pattern is not None:
        c_theta, c_phi = second_pattern(theta, np.zeros_like(theta))
        g = np.abs(c_theta) ** 2 + np.abs(c_phi) ** 2
        g = np.where(g == 0, 1e-12, g)
        g_db = 10 * np.log10(g)
        g_db_max = np.max(g_db)
        g_db_min = np.min(g_db)
        if g_db_min == g_db_max:
            g_db_min = -30
        else:
            g_db_min = np.maximum(-60., g_db_min)
        
        plt.polar(theta, g_db, label='Test pattern', linestyle='--')

    fig_v.axes[0].set_rmin(-60)
    fig_v.axes[0].set_rmax(5)
    fig_v.axes[0].set_theta_zero_location("N")
    fig_v.axes[0].set_theta_direction(-1)
    plt.title(r"Vertical cut of the radiation pattern $G(\theta,0)$ ")
    plt.legend()
    return fig_v

In [ ]:
model

In [ ]:
visualize_vertical(model, tr38901_pattern);
visualize_horizzontal(model, tr38901_pattern);

In [ ]:
from sionna.rt.antenna import visualize
visualize(hw_dipole_pattern);
visualize(model)

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sionna.rt import polarization_model_2
import numpy as np
def plot_polar(data, title):
    theta = [x * (3.141592653589793 / 180) for x in data['x']]  # Convert degrees to radians
    # Step 1: Find the maximum gain
    r = data['y']
    max_gain = max(r)

    # Step 2: Rescale the antenna pattern
    rescaled_pattern = [gain - max_gain for gain in r]
    c = tf.complex(tf.sqrt(r), tf.zeros_like(r))[:10]
    t,p = polarization_model_2(c, 0.0)
    g = np.abs(t)**2 + np.abs(p)**2
   
    # Step 2: Rescale the antenna pattern
    rescaled_pattern_2 = [gain - max(g) for gain in g]
    print("------\n",t,p,g, rescaled_pattern_2)

    plt.figure()
    ax = plt.subplot(111, polar=True)
    ax.plot(theta, rescaled_pattern)
    ax.set_title(title, va='bottom')
    ax.set_ylim([-100, 0])
    plt.show()

    # plt.figure()
    # ax = plt.subplot(111, polar=True)
    # ax.plot(theta, rescaled_pattern_2)
    # ax.set_title(title, va='bottom')
    # ax.set_ylim([-100, 0])
    # plt.show()

In [ ]:
plot_polar(horizontal_data, 'Horizontal Data Polar Plot')
plot_polar(vertical_data, 'Vertical Data Polar Plot')